In [1]:
!python -m pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 36.6 MB/s eta 0:00:00


https://ftp.ncbi.nlm.nih.gov/pubchem/Compound/

https://pubchem.ncbi.nlm.nih.gov/

https://chembl.gitbook.io/chembl-interface-documentation/downloads

https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/

In [ ]:
#from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs

def perform_similarity_search(compound_smiles, chembl_compounds, threshold=0.3):
    query_mol = Chem.MolFromSmiles(compound_smiles)
    query_fp = MACCSkeys.GenMACCSKeys(query_mol)

    similar_compounds = []
    # Iterate over compounds in ChEMBL and calculate similarity
    for molecule in chembl_compounds:
        try:
            smiles = molecule['molecule_structures']['canonical_smiles']
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                fp = MACCSkeys.GenMACCSKeys(mol)
                similarity = DataStructs.FingerprintSimilarity(query_fp, fp)
                if similarity >= threshold:
                    similar_compounds.append(molecule['molecule_chembl_id'])
        except KeyError:
            continue  # Skip this molecule if structure information is not available

    return similar_compounds


In [ ]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys, DataStructs

# Define a sample query compound SMILES string
query_smiles = 'CCO'

# Define a sample ChEMBL compounds list (just for demonstration)
chembl_compounds = [
    {'molecule_structures': {'canonical_smiles': 'CCOCO'}, 'molecule_chembl_id': 'CHEMBL1'},
    {'molecule_structures': {'canonical_smiles': 'CCN'}, 'molecule_chembl_id': 'CHEMBL2'},
    {'molecule_structures': {'canonical_smiles': 'CCC'}, 'molecule_chembl_id': 'CHEMBL3'}
]

# Define a threshold for similarity
threshold = 0.4

# Perform similarity search
similar_compounds = perform_similarity_search(query_smiles, chembl_compounds, threshold)

# Print the similar compounds found
print("Similar compounds:")
for compound in similar_compounds:
    print(compound)

Similar compounds:
CHEMBL1


In [1]:
!pip install chembl-webresource-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.2.0
    Uninstalling attrs-23.2.0:
      Successfully uninstalled attrs-23.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jsonschema 4.19.2 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.
referencing 0.33.0 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.


https://mybinder.org/v2/gh/chembl/chembl_webresource_client/master?filepath=demo_wrc.ipynb

In [2]:
from chembl_webresource_client.new_client import new_client

similarity = new_client.similarity
res = similarity.filter(smiles="CO[C@@H](CCC#C\C=C/CCCC(C)CCCCC=C)C(=O)[O-]", similarity=70).only(['molecule_chembl_id', 'similarity'])
for i in res:
    print(i)

{'molecule_chembl_id': 'CHEMBL478779', 'similarity': '85.4166686534881591796875'}
{'molecule_chembl_id': 'CHEMBL477889', 'similarity': '85.4166686534881591796875'}
{'molecule_chembl_id': 'CHEMBL477888', 'similarity': '85.4166686534881591796875'}
{'molecule_chembl_id': 'CHEMBL2304268', 'similarity': '70.1754391193389892578125'}


In [3]:
from chembl_webresource_client.new_client import new_client

def perform_similarity_search(smiles, threshold=60):
    similarity = new_client.similarity
    # Perform similarity search using ChEMBL's similarity search API
    res = similarity.filter(smiles=smiles, similarity=threshold).only(['molecule_chembl_id', 'similarity'])
    similar_compounds = [(entry['molecule_chembl_id'],round(float(entry['similarity']),2)) for entry in res]
    similar_compounds_ = [entry['molecule_chembl_id'] for entry in res]
    return similar_compounds_

# Screening compounds with their IDs and SMILES strings
screening_compounds = {
    'NCGC00602080-03': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=C(F)C=CC=C4)N=C2C',
    'NCGC00685598-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC=CC=C4)N=C2C',
    'NCGC00685517-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC=CC(F)=C4)N=C2C',
    'NCGC00686269-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC(F)=CC(F)=C4)N=C2C'}

# Perform chemical structure similarity search against ChEMBL for each screening compound
similar_compounds_dict = {}
for compound_id, compound_smiles in screening_compounds.items():
    similar_compounds = perform_similarity_search(compound_smiles)
    print(similar_compounds)
    similar_compounds_dict[compound_id] = similar_compounds

similar_compounds_dict

['CHEMBL1486414', 'CHEMBL1599454', 'CHEMBL1314806']
['CHEMBL1409278', 'CHEMBL1437723', 'CHEMBL1894934', 'CHEMBL1522660', 'CHEMBL1360118', 'CHEMBL1571836', 'CHEMBL1532410', 'CHEMBL1395809', 'CHEMBL1700697', 'CHEMBL1419234']
['CHEMBL1430536', 'CHEMBL1409278', 'CHEMBL1358484']
['CHEMBL1360118']


{'NCGC00602080-03': ['CHEMBL1486414', 'CHEMBL1599454', 'CHEMBL1314806'],
 'NCGC00685598-01': ['CHEMBL1409278',
  'CHEMBL1437723',
  'CHEMBL1894934',
  'CHEMBL1522660',
  'CHEMBL1360118',
  'CHEMBL1571836',
  'CHEMBL1532410',
  'CHEMBL1395809',
  'CHEMBL1700697',
  'CHEMBL1419234'],
 'NCGC00685517-01': ['CHEMBL1430536', 'CHEMBL1409278', 'CHEMBL1358484'],
 'NCGC00686269-01': ['CHEMBL1360118']}

In [ ]:
from chembl_webresource_client.new_client import new_client

def perform_connectivity_search(smiles, threshold=60):
    molecule = new_client.molecule
    # Perform similarity search using ChEMBL's similarity search API
    res = molecule.filter(molecule_structures__canonical_smiles__connectivity=smiles).only(['molecule_chembl_id', 'pref_name'])
    similar_compounds = [(entry['molecule_chembl_id'],entry['pref_name']) for entry in res]
    return similar_compounds

# Screening compounds with their IDs and SMILES strings
screening_compounds = {
    'NCGC00602080-03': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=C(F)C=CC=C4)N=C2C',
    'NCGC00685598-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC=CC=C4)N=C2C',
    'NCGC00685517-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC=CC(F)=C4)N=C2C',
    'NCGC00686269-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC(F)=CC(F)=C4)N=C2C'}

# Perform chemical structure similarity search against ChEMBL for each screening compound
similar_compounds_dict = {}
for compound_id, compound_smiles in screening_compounds.items():
    similar_compounds = perform_connectivity_search(compound_smiles)
    print(similar_compounds)
    similar_compounds_dict[compound_id] = similar_compounds

[]
[]
[]
[]


In [ ]:
from chembl_webresource_client.new_client import new_client

target = new_client.target
gene_name = 'BRD4'
res = target.filter(target_synonym__icontains=gene_name).only(['organism', 'pref_name', 'target_type'])
for i in res:
    print(i)


{'organism': 'Homo sapiens', 'pref_name': 'Bromodomain-containing protein 4', 'target_type': 'SINGLE PROTEIN'}
{'organism': 'Mus musculus', 'pref_name': 'Bromodomain-containing protein 4', 'target_type': 'SINGLE PROTEIN'}
{'organism': 'Homo sapiens', 'pref_name': 'BRD4/HDAC1', 'target_type': 'PROTEIN COMPLEX'}
{'organism': 'Homo sapiens', 'pref_name': 'Cereblon/Cullin-4A/Bromodomain-containing protein 4', 'target_type': 'PROTEIN-PROTEIN INTERACTION'}
{'organism': 'Homo sapiens', 'pref_name': 'Cereblon/Bromodomain-containing protein 4', 'target_type': 'PROTEIN-PROTEIN INTERACTION'}
{'organism': 'Homo sapiens', 'pref_name': 'von Hippel-Lindau disease tumor suppressor/Bromodomain-containing protein 4', 'target_type': 'PROTEIN-PROTEIN INTERACTION'}
{'organism': 'Homo sapiens', 'pref_name': 'Cereblon/DNA damage-binding protein 1/Bromodomain-containing protein 4', 'target_type': 'PROTEIN-PROTEIN INTERACTION'}
{'organism': 'Homo sapiens', 'pref_name': 'von Hippel-Lindau disease tumor suppress

In [ ]:
from chembl_webresource_client.new_client import new_client
target = new_client.target
gene_name = 'CHEMBL1829'
res = target.filter(target=gene_name).only(['organism', 'pref_name', 'target_type'])
for i in res:
    print(i)

In [ ]:
from chembl_webresource_client.new_client import new_client

def get_targets(chembl_ids):
    targets = set()
    target_client = new_client.target
    for chembl_id in chembl_ids:
        # Filter targets associated with the ChEMBL compound
        res = target_client.filter(target=chembl_id).only(['organism', 'pref_name', 'target_type'])
        for entry in res:
            targets.add((entry['organism'], entry['pref_name'], entry['target_type']))
    return targets

# Define the TOR1A target
TOR1A = 'CHEMBL1829'

# Get targets associated with ChEMBL compounds
chembl_targets = set()
for similar_compounds in similar_compounds_dict.values():
    chembl_targets.update(get_targets(similar_compounds))

# Compare ChEMBL targets with TOR1A to identify shared targets
shared_targets = [target for target in chembl_targets if target[0] == 'Homo sapiens' and target[1] == 'TOR1A']

# Print the shared targets
print("Shared targets associated with DYT1 (TOR1A):")
for target in shared_targets:
    print(target)

In [4]:
from chembl_webresource_client.new_client import new_client

def get_targets(chembl_ids):
    targets = set()
    target_client = new_client.target
    for chembl_id in chembl_ids:
        # Filter targets associated with the ChEMBL compound
        res = target_client.filter(target=chembl_id).only(['organism', 'pref_name', 'target_type'])
        for entry in res:
            targets.add((entry['organism'], entry['pref_name'], entry['target_type']))
    return targets

# Define the TOR1A target
TOR1A = 'CHEMBL1829'

# Get targets associated with ChEMBL compounds
chembl_targets = set()
for similar_compounds in similar_compounds_dict.values():
    chembl_targets.update(get_targets(similar_compounds))

# Compare ChEMBL targets with TOR1A to identify shared targets
shared_targets = [target for target in chembl_targets if target[0] == 'Homo sapiens' and target[1] == 'TOR1A']

# Print the shared targets
print("Shared targets associated with DYT1 (TOR1A):")
for target in shared_targets:
    print(target)

In [8]:
from chembl_webresource_client.new_client import new_client

# Function to retrieve targets associated with a ChEMBL ID
def get_targets(chembl_id):
    target_client = new_client.target
    # Retrieve targets associated with the ChEMBL ID
    res = target_client.filter(target=chembl_id).only(['target_chembl_id','organism','pref_name'])
    res = res[:100]
    # Extract the target ChEMBL IDs
    return {(entry['organism'],entry['target_chembl_id'], entry['pref_name']) for entry in res}


# Function to find shared targets between two ChEMBL IDs
def find_shared_targets(chembl_id1, chembl_ids):
    # Retrieve targets associated with each ChEMBL ID
    targets1 = get_targets(chembl_id1)
    # Find the intersection of targets to get shared targets
    shared_targets = set()
    for chembl_id in chembl_ids:
        targets2 = get_targets(chembl_id)
        shared_targets.update(targets1.intersection(targets2))
    return shared_targets

# Get targets associated with ChEMBL compounds
chembl_targets = set()
for similar_compounds in similar_compounds_dict.values():
    chembl_targets.update(similar_compounds)

TOR1A = 'CHEMBL1829'  # Define the TOR1A target

# Find shared targets
shared_targets = find_shared_targets(TOR1A, chembl_targets)
print("Shared Targets:", list(shared_targets))  # Convert set to list for printing

Shared Targets: [('Homo sapiens', 'CHEMBL223', 'Alpha-1d adrenergic receptor'), ('Homo sapiens', 'CHEMBL1808', 'Angiotensin-converting enzyme'), ('Homo sapiens', 'CHEMBL1908', 'Cytochrome P450 11B1'), ('Homo sapiens', 'CHEMBL1994', 'Mineralocorticoid receptor'), ('Homo sapiens', 'CHEMBL1830', 'Ribonucleoside-diphosphate reductase M1 chain'), ('Homo sapiens', 'CHEMBL203', 'Epidermal growth factor receptor erbB1'), ('Homo sapiens', 'CHEMBL221', 'Cyclooxygenase-1'), ('Homo sapiens', 'CHEMBL1842', 'Myeloid cell surface antigen CD33'), ('Homo sapiens', 'CHEMBL1860', 'Thyroid hormone receptor alpha'), ('Homo sapiens', 'CHEMBL1839', 'Thyroid peroxidase'), ('Influenza A virus (A/Udorn/307/1972(H3N2))', 'CHEMBL2052', 'Influenza virus A matrix protein M2'), ('Homo sapiens', 'CHEMBL1790', 'Vasopressin V2 receptor'), ('Homo sapiens', 'CHEMBL1781', 'DNA topoisomerase I'), ('Homo sapiens', 'CHEMBL1951', 'Monoamine oxidase A'), ('Homo sapiens', 'CHEMBL1983', 'Serotonin 1d (5-HT1d) receptor'), ('Homo 

https://hpo.jax.org/app/browse/disease/ORPHA:98807

https://hpo.jax.org/app/browse/gene/1861

In [ ]:
https://hpo.jax.org/app/browse/disease/ORPHA:98807